In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving combined_matrix.csv to combined_matrix.csv


In [ ]:
import pandas as pd
import numpy as np
import keras
from keras import layers
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import random
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
sorted_combined_matrix = pd.read_csv("combined_matrix.csv")

In [ ]:
sorted_combined_matrix

,parameter_location_id,2019-10-28T02:00:00+08:00,2018-05-28T12:00:00+08:00,2018-02-23T17:00:00+08:00,2018-03-25T14:00:00+08:00,2019-11-14T11:00:00+08:00,2021-06-01T17:00:00+08:00,2018-09-09T12:00:00+08:00,2018-12-04T01:00:00+08:00,2019-05-22T20:00:00+08:00,...,2019-08-13T15:00:00+08:00,2020-12-07T18:00:00+08:00,2021-01-15T20:00:00+08:00,2021-03-24T13:00:00+08:00,2018-08-17T02:00:00+08:00,2020-02-08T22:00:00+08:00,2019-04-18T07:00:00+08:00,2020-12-28T06:00:00+08:00,2020-04-14T11:00:00+08:00,2021-01-09T12:00:00+08:00
0,co_6168,1400.0,769.0,680.0,796.0,400.0,700.0,363.0,2000.0,800.0,...,400.0,800.0,1100.0,500.0,752.0,300.0,800.0,1100.0,500.0,500.0
1,no2_6168,73.0,65.0,44.0,49.0,55.0,22.0,14.0,45.0,97.0,...,22.0,63.0,89.0,36.0,12.0,17.0,80.0,68.0,46.0,29.0
2,o3_6168,49.0,78.0,76.0,113.0,63.0,243.0,130.0,21.0,199.0,...,182.0,49.0,49.0,91.0,51.0,60.0,137.0,37.0,131.0,53.0
3,pm10_6168,141.0,187.0,45.0,0.0,99.0,63.0,33.0,0.0,113.0,...,27.0,72.0,230.0,63.0,35.0,26.0,84.0,58.0,55.0,36.0
4,pm25_6168,80.0,57.0,29.0,103.0,32.0,24.0,8.0,25.0,29.0,...,10.0,70.0,87.0,14.0,27.0,22.0,13.0,82.0,46.0,18.0
5,so2_6168,10.0,14.0,5.0,15.0,7.0,8.0,3.0,3.0,4.0,...,4.0,2.0,16.0,8.0,7.0,4.0,5.0,1.0,9.0,8.0
6,co_6169,1400.0,809.0,905.0,630.0,500.0,600.0,566.0,1900.0,1000.0,...,600.0,800.0,1000.0,600.0,596.0,500.0,800.0,800.0,400.0,400.0
7,no2_6169,61.0,50.0,23.0,23.0,41.0,24.0,20.0,36.0,51.0,...,11.0,45.0,77.0,27.0,11.0,22.0,40.0,62.0,29.0,19.0
8,o3_6169,52.0,91.0,99.0,145.0,68.0,243.0,143.0,15.0,170.0,...,146.0,58.0,56.0,114.0,56.0,62.0,138.0,47.0,142.0,56.0
9,pm10_6169,105.0,202.0,0.0,65.0,60.0,83.0,42.0,0.0,141.0,...,39.0,67.0,203.0,61.0,38.0,28.0,59.0,72.0,68.0,32.0


In [ ]:
sorted_combined_matrix = sorted_combined_matrix.set_index('parameter_location_id')

sorted_combined_matrix_T =sorted_combined_matrix.T
sorted_combined_matrix_T.replace(to_replace=0, method='ffill', inplace=True)
sorted_combined_matrix_T.index

Index(['2019-10-28T02:00:00+08:00', '2018-05-28T12:00:00+08:00',
       '2018-02-23T17:00:00+08:00', '2018-03-25T14:00:00+08:00',
       '2019-11-14T11:00:00+08:00', '2021-06-01T17:00:00+08:00',
       '2018-09-09T12:00:00+08:00', '2018-12-04T01:00:00+08:00',
       '2019-05-22T20:00:00+08:00', '2020-05-07T23:00:00+08:00',
       ...
       '2019-08-13T15:00:00+08:00', '2020-12-07T18:00:00+08:00',
       '2021-01-15T20:00:00+08:00', '2021-03-24T13:00:00+08:00',
       '2018-08-17T02:00:00+08:00', '2020-02-08T22:00:00+08:00',
       '2019-04-18T07:00:00+08:00', '2020-12-28T06:00:00+08:00',
       '2020-04-14T11:00:00+08:00', '2021-01-09T12:00:00+08:00'],
      dtype='object', length=25941)

In [ ]:
# Normalize the combined_matrix data
normalized_data = sorted_combined_matrix_T
scaler = StandardScaler()
scaled_dataset = scaler.fit_transform(normalized_data)

In [ ]:
optimizers = ['adam', 'rmsprop']
activation_functions = ['relu', 'sigmoid', 'tanh', 'elu', 'selu', 'softmax', 'swish']
mse_values = []
window_size = 3

In [ ]:
# Define a function to create supervised data
def to_supervised(train):
    X = []
    Y = []
    for i in range(window_size, len(train)):
        X.append(train[i - window_size:i, :])
        Y.append(train[i, 0:1])
    return X, Y

In [ ]:

for optimizer in optimizers:
    for activation in activation_functions:
        # Load or create your 'scaled_dataset' here

        # Create supervised data
        X, Y = to_supervised(scaled_dataset)
        X = np.array(X)
        Y = np.array(Y)

        # Split the data into training and testing sets
        test_size = 0.2
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=42)

        # Build the LSTM model
        model = Sequential()
        model.add(LSTM(units=100, return_sequences=True, input_shape=(window_size, X_train.shape[2]), activation=activation))
        model.add(LSTM(units=100, return_sequences=True, activation=activation))
        model.add(LSTM(units=100, activation=activation))
        model.add(Dense(units=100))
        model.add(Dense(units=X_train.shape[2]))
        model.compile(optimizer=optimizer, loss='mean_squared_error')

        # Train the model
        lstm_history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=100, batch_size=256, verbose=2)

        # Make predictions and calculate MSE
        y_pred = model.predict(X_test)
        y_pred_reduced = y_pred.mean(axis=1)
        mse = mean_squared_error(Y_test, y_pred_reduced)

        print("Average MSE for the " + str(optimizer) + " optimizer and " + str(activation) + " activation function:", mse)
        mse_values.append(mse)

Epoch 1/100
82/82 - 6s - loss: 0.9761 - val_loss: 1.0962 - 6s/epoch - 73ms/step
Epoch 2/100
82/82 - 1s - loss: 0.9759 - val_loss: 1.1043 - 1s/epoch - 17ms/step
Epoch 3/100
82/82 - 1s - loss: 0.9763 - val_loss: 1.0961 - 1s/epoch - 17ms/step
Epoch 4/100
82/82 - 1s - loss: 0.9752 - val_loss: 1.0961 - 1s/epoch - 17ms/step
Epoch 5/100
82/82 - 1s - loss: 0.9744 - val_loss: 1.1015 - 1s/epoch - 17ms/step
Epoch 6/100
82/82 - 1s - loss: 0.9724 - val_loss: 1.0996 - 1s/epoch - 17ms/step
Epoch 7/100
82/82 - 1s - loss: 0.9696 - val_loss: 1.0984 - 1s/epoch - 18ms/step
Epoch 8/100
82/82 - 2s - loss: 0.9667 - val_loss: 1.1000 - 2s/epoch - 18ms/step
Epoch 9/100
82/82 - 1s - loss: 0.9619 - val_loss: 1.1180 - 1s/epoch - 17ms/step
Epoch 10/100
82/82 - 1s - loss: 0.9564 - val_loss: 1.1197 - 1s/epoch - 17ms/step
Epoch 11/100
82/82 - 1s - loss: 0.9451 - val_loss: 1.1264 - 1s/epoch - 17ms/step
Epoch 12/100
82/82 - 1s - loss: 0.9368 - val_loss: 1.1263 - 1s/epoch - 17ms/step
Epoch 13/100
82/82 - 1s - loss: 0.917

Epoch 1/100
82/82 - 7s - loss: 0.9839 - val_loss: 1.0955 - 7s/epoch - 83ms/step
Epoch 2/100
82/82 - 1s - loss: 0.9765 - val_loss: 1.0973 - 1s/epoch - 17ms/step
Epoch 3/100
82/82 - 1s - loss: 0.9770 - val_loss: 1.0963 - 1s/epoch - 17ms/step
Epoch 4/100
82/82 - 1s - loss: 0.9768 - val_loss: 1.0967 - 1s/epoch - 17ms/step
Epoch 5/100
82/82 - 1s - loss: 0.9767 - val_loss: 1.0969 - 1s/epoch - 16ms/step
Epoch 6/100
82/82 - 1s - loss: 0.9768 - val_loss: 1.0966 - 1s/epoch - 16ms/step
Epoch 7/100
82/82 - 1s - loss: 0.9762 - val_loss: 1.0956 - 1s/epoch - 16ms/step
Epoch 8/100
82/82 - 1s - loss: 0.9764 - val_loss: 1.0957 - 1s/epoch - 16ms/step
Epoch 9/100
82/82 - 1s - loss: 0.9764 - val_loss: 1.0959 - 1s/epoch - 16ms/step
Epoch 10/100
82/82 - 1s - loss: 0.9761 - val_loss: 1.0959 - 1s/epoch - 16ms/step
Epoch 11/100
82/82 - 1s - loss: 0.9762 - val_loss: 1.0958 - 1s/epoch - 17ms/step
Epoch 12/100
82/82 - 1s - loss: 0.9762 - val_loss: 1.0962 - 1s/epoch - 17ms/step
Epoch 13/100
82/82 - 1s - loss: 0.976

Epoch 1/100
82/82 - 6s - loss: 0.9762 - val_loss: 1.0956 - 6s/epoch - 73ms/step
Epoch 2/100
82/82 - 1s - loss: 0.9756 - val_loss: 1.0959 - 1s/epoch - 17ms/step
Epoch 3/100
82/82 - 1s - loss: 0.9751 - val_loss: 1.0969 - 1s/epoch - 16ms/step
Epoch 4/100
82/82 - 1s - loss: 0.9747 - val_loss: 1.0970 - 1s/epoch - 16ms/step
Epoch 5/100
82/82 - 1s - loss: 0.9731 - val_loss: 1.0963 - 1s/epoch - 17ms/step
Epoch 6/100
82/82 - 1s - loss: 0.9715 - val_loss: 1.0954 - 1s/epoch - 17ms/step
Epoch 7/100
82/82 - 1s - loss: 0.9709 - val_loss: 1.0956 - 1s/epoch - 17ms/step
Epoch 8/100
82/82 - 1s - loss: 0.9683 - val_loss: 1.0987 - 1s/epoch - 17ms/step
Epoch 9/100
82/82 - 1s - loss: 0.9629 - val_loss: 1.1197 - 1s/epoch - 17ms/step
Epoch 10/100
82/82 - 1s - loss: 0.9598 - val_loss: 1.1052 - 1s/epoch - 18ms/step
Epoch 11/100
82/82 - 1s - loss: 0.9501 - val_loss: 1.1241 - 1s/epoch - 17ms/step
Epoch 12/100
82/82 - 1s - loss: 0.9399 - val_loss: 1.1303 - 1s/epoch - 16ms/step
Epoch 13/100
82/82 - 1s - loss: 0.924

Epoch 1/100
82/82 - 6s - loss: 0.9773 - val_loss: 1.0959 - 6s/epoch - 76ms/step
Epoch 2/100
82/82 - 1s - loss: 0.9762 - val_loss: 1.0954 - 1s/epoch - 17ms/step
Epoch 3/100
82/82 - 1s - loss: 0.9761 - val_loss: 1.0962 - 1s/epoch - 17ms/step
Epoch 4/100
82/82 - 1s - loss: 0.9746 - val_loss: 1.0959 - 1s/epoch - 17ms/step
Epoch 5/100
82/82 - 1s - loss: 0.9729 - val_loss: 1.0968 - 1s/epoch - 18ms/step
Epoch 6/100
82/82 - 1s - loss: 0.9706 - val_loss: 1.0997 - 1s/epoch - 17ms/step
Epoch 7/100
82/82 - 1s - loss: 0.9690 - val_loss: 1.0996 - 1s/epoch - 17ms/step
Epoch 8/100
82/82 - 1s - loss: 0.9628 - val_loss: 1.1004 - 1s/epoch - 17ms/step
Epoch 9/100
82/82 - 1s - loss: 0.9581 - val_loss: 1.1061 - 1s/epoch - 17ms/step
Epoch 10/100
82/82 - 1s - loss: 0.9458 - val_loss: 1.1253 - 1s/epoch - 17ms/step
Epoch 11/100
82/82 - 1s - loss: 0.9320 - val_loss: 1.1253 - 1s/epoch - 17ms/step
Epoch 12/100
82/82 - 1s - loss: 0.9027 - val_loss: 1.1327 - 1s/epoch - 17ms/step
Epoch 13/100
82/82 - 1s - loss: 0.873

Epoch 1/100
82/82 - 6s - loss: 0.9762 - val_loss: 1.0955 - 6s/epoch - 75ms/step
Epoch 2/100
82/82 - 1s - loss: 0.9760 - val_loss: 1.0959 - 1s/epoch - 17ms/step
Epoch 3/100
82/82 - 1s - loss: 0.9760 - val_loss: 1.0959 - 1s/epoch - 17ms/step
Epoch 4/100
82/82 - 1s - loss: 0.9761 - val_loss: 1.0961 - 1s/epoch - 17ms/step
Epoch 5/100
82/82 - 1s - loss: 0.9760 - val_loss: 1.0960 - 1s/epoch - 18ms/step
Epoch 6/100
82/82 - 1s - loss: 0.9760 - val_loss: 1.0958 - 1s/epoch - 18ms/step
Epoch 7/100
82/82 - 1s - loss: 0.9760 - val_loss: 1.0960 - 1s/epoch - 18ms/step
Epoch 8/100
82/82 - 1s - loss: 0.9760 - val_loss: 1.0959 - 1s/epoch - 18ms/step
Epoch 9/100
82/82 - 1s - loss: 0.9760 - val_loss: 1.0958 - 1s/epoch - 17ms/step
Epoch 10/100
82/82 - 1s - loss: 0.9760 - val_loss: 1.0959 - 1s/epoch - 17ms/step
Epoch 11/100
82/82 - 1s - loss: 0.9761 - val_loss: 1.0960 - 1s/epoch - 17ms/step
Epoch 12/100
82/82 - 1s - loss: 0.9760 - val_loss: 1.0959 - 1s/epoch - 17ms/step
Epoch 13/100
82/82 - 1s - loss: 0.976

...

Epoch 1/100
82/82 - 5s - loss: 0.9829 - val_loss: 1.1209 - 5s/epoch - 67ms/step
Epoch 2/100
82/82 - 1s - loss: 0.9788 - val_loss: 1.1122 - 1s/epoch - 17ms/step
Epoch 3/100
82/82 - 1s - loss: 0.9779 - val_loss: 1.1188 - 1s/epoch - 17ms/step
Epoch 4/100
82/82 - 1s - loss: 0.9775 - val_loss: 1.0988 - 1s/epoch - 16ms/step
Epoch 5/100
82/82 - 1s - loss: 0.9771 - val_loss: 1.1001 - 1s/epoch - 17ms/step
Epoch 6/100
82/82 - 1s - loss: 0.9767 - val_loss: 1.0972 - 1s/epoch - 17ms/step
Epoch 7/100
82/82 - 1s - loss: 0.9768 - val_loss: 1.0999 - 1s/epoch - 16ms/step
Epoch 8/100
82/82 - 1s - loss: 0.9766 - val_loss: 1.0963 - 1s/epoch - 17ms/step
Epoch 9/100
82/82 - 1s - loss: 0.9765 - val_loss: 1.0975 - 1s/epoch - 16ms/step
Epoch 10/100
82/82 - 1s - loss: 0.9764 - val_loss: 1.0964 - 1s/epoch - 17ms/step
Epoch 11/100
82/82 - 1s - loss: 0.9763 - val_loss: 1.1008 - 1s/epoch - 17ms/step
Epoch 12/100
82/82 - 1s - loss: 0.9764 - val_loss: 1.0963 - 1s/epoch - 17ms/step
Epoch 13/100
82/82 - 1s - loss: 0.976

Epoch 1/100
82/82 - 6s - loss: 0.9763 - val_loss: 1.0955 - 6s/epoch - 69ms/step
Epoch 2/100
82/82 - 1s - loss: 0.9761 - val_loss: 1.0953 - 1s/epoch - 18ms/step
Epoch 3/100
82/82 - 1s - loss: 0.9760 - val_loss: 1.0958 - 1s/epoch - 17ms/step
Epoch 4/100
82/82 - 1s - loss: 0.9759 - val_loss: 1.0953 - 1s/epoch - 16ms/step
Epoch 5/100
82/82 - 1s - loss: 0.9757 - val_loss: 1.0957 - 1s/epoch - 16ms/step
Epoch 6/100
82/82 - 1s - loss: 0.9755 - val_loss: 1.0981 - 1s/epoch - 17ms/step
Epoch 7/100
82/82 - 1s - loss: 0.9754 - val_loss: 1.0974 - 1s/epoch - 16ms/step
Epoch 8/100
82/82 - 1s - loss: 0.9751 - val_loss: 1.0957 - 1s/epoch - 17ms/step
Epoch 9/100
82/82 - 1s - loss: 0.9746 - val_loss: 1.0983 - 1s/epoch - 17ms/step
Epoch 10/100
82/82 - 1s - loss: 0.9741 - val_loss: 1.0956 - 1s/epoch - 17ms/step
Epoch 11/100
82/82 - 1s - loss: 0.9730 - val_loss: 1.0968 - 1s/epoch - 18ms/step
Epoch 12/100
82/82 - 1s - loss: 0.9731 - val_loss: 1.0987 - 1s/epoch - 17ms/step
Epoch 13/100
82/82 - 1s - loss: 0.972

Epoch 1/100
82/82 - 5s - loss: 0.9776 - val_loss: 1.0961 - 5s/epoch - 66ms/step
Epoch 2/100
82/82 - 1s - loss: 0.9763 - val_loss: 1.0956 - 1s/epoch - 16ms/step
Epoch 3/100
82/82 - 1s - loss: 0.9753 - val_loss: 1.0985 - 1s/epoch - 16ms/step
Epoch 4/100
82/82 - 1s - loss: 0.9748 - val_loss: 1.0954 - 1s/epoch - 16ms/step
Epoch 5/100
82/82 - 1s - loss: 0.9736 - val_loss: 1.0971 - 1s/epoch - 16ms/step
Epoch 6/100
82/82 - 1s - loss: 0.9730 - val_loss: 1.0980 - 1s/epoch - 16ms/step
Epoch 7/100
82/82 - 1s - loss: 0.9715 - val_loss: 1.1045 - 1s/epoch - 17ms/step
Epoch 8/100
82/82 - 1s - loss: 0.9700 - val_loss: 1.0982 - 1s/epoch - 17ms/step
Epoch 9/100
82/82 - 1s - loss: 0.9678 - val_loss: 1.1044 - 1s/epoch - 16ms/step
Epoch 10/100
82/82 - 1s - loss: 0.9669 - val_loss: 1.1028 - 1s/epoch - 16ms/step
Epoch 11/100
82/82 - 1s - loss: 0.9646 - val_loss: 1.1151 - 1s/epoch - 16ms/step
Epoch 12/100
82/82 - 1s - loss: 0.9628 - val_loss: 1.0996 - 1s/epoch - 16ms/step
Epoch 13/100
82/82 - 1s - loss: 0.958

Epoch 1/100
82/82 - 6s - loss: 0.9761 - val_loss: 1.0954 - 6s/epoch - 68ms/step
Epoch 2/100
82/82 - 1s - loss: 0.9760 - val_loss: 1.0966 - 1s/epoch - 18ms/step
Epoch 3/100
82/82 - 1s - loss: 0.9761 - val_loss: 1.0962 - 1s/epoch - 18ms/step
Epoch 4/100
82/82 - 1s - loss: 0.9761 - val_loss: 1.0966 - 1s/epoch - 18ms/step
Epoch 5/100
82/82 - 1s - loss: 0.9761 - val_loss: 1.0957 - 1s/epoch - 17ms/step
Epoch 6/100
82/82 - 1s - loss: 0.9761 - val_loss: 1.0958 - 1s/epoch - 18ms/step
Epoch 7/100
82/82 - 1s - loss: 0.9760 - val_loss: 1.0954 - 1s/epoch - 17ms/step
Epoch 8/100
82/82 - 1s - loss: 0.9762 - val_loss: 1.0959 - 1s/epoch - 17ms/step
Epoch 9/100
82/82 - 1s - loss: 0.9761 - val_loss: 1.0957 - 1s/epoch - 17ms/step
Epoch 10/100
82/82 - 1s - loss: 0.9760 - val_loss: 1.0959 - 1s/epoch - 17ms/step
Epoch 11/100
82/82 - 1s - loss: 0.9761 - val_loss: 1.0956 - 1s/epoch - 17ms/step
Epoch 12/100
82/82 - 1s - loss: 0.9761 - val_loss: 1.0954 - 1s/epoch - 18ms/step
Epoch 13/100
82/82 - 1s - loss: 0.976

Epoch 1/100
82/82 - 7s - loss: 0.9761 - val_loss: 1.0958 - 7s/epoch - 89ms/step
Epoch 2/100
82/82 - 2s - loss: 0.9761 - val_loss: 1.0955 - 2s/epoch - 19ms/step
Epoch 3/100
82/82 - 2s - loss: 0.9761 - val_loss: 1.0961 - 2s/epoch - 19ms/step
Epoch 4/100
82/82 - 2s - loss: 0.9761 - val_loss: 1.0968 - 2s/epoch - 20ms/step
Epoch 5/100
82/82 - 2s - loss: 0.9760 - val_loss: 1.0974 - 2s/epoch - 19ms/step
Epoch 6/100
82/82 - 1s - loss: 0.9761 - val_loss: 1.0957 - 1s/epoch - 18ms/step
Epoch 7/100
82/82 - 1s - loss: 0.9760 - val_loss: 1.0958 - 1s/epoch - 18ms/step
Epoch 8/100
82/82 - 1s - loss: 0.9761 - val_loss: 1.0964 - 1s/epoch - 18ms/step
Epoch 9/100
82/82 - 1s - loss: 0.9761 - val_loss: 1.0959 - 1s/epoch - 18ms/step
Epoch 10/100
82/82 - 1s - loss: 0.9761 - val_loss: 1.0963 - 1s/epoch - 18ms/step
Epoch 11/100
82/82 - 1s - loss: 0.9760 - val_loss: 1.0962 - 1s/epoch - 18ms/step
Epoch 12/100
82/82 - 2s - loss: 0.9761 - val_loss: 1.0969 - 2s/epoch - 19ms/step
Epoch 13/100
82/82 - 2s - loss: 0.976

In [ ]:
print(mse_values)


[1.6985966147239215, 1.2893571279908824, 1.8843867844482098, 1.754063754330002, 1.584303399304113, 1.0959533066900087, 1.7860286924905349, 1.567491279898536, 1.096217260216236, 1.6995887848253155, 1.667131366955679, 1.6487379253169203, 1.0960054157654937, 1.0960785395736907]
